## Importing Libraries

In [ ]:
!pip install --upgrade pip
!pip install mediapipe-model-maker

In [ ]:
from google.colab import files
import os
import tensorflow as tf
assert tf.__version__.startswith('2')

from mediapipe_model_maker import image_classifier

import matplotlib.pyplot as plt

## Loading Dataset

In [ ]:
#save final weights to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
zip_file_name = "/content/drive/MyDrive/FruitsDataset" #@param {type:"string"}
!unzip {zip_file_name}.zip

Archive:  /content/drive/MyDrive/2023FlutterML/FruitsDataset.zip
   creating: FruitsDataset/
   creating: FruitsDataset/bananas/
   creating: FruitsDataset/pineapples/
  inflating: FruitsDataset/.DS_Store  
  inflating: __MACOSX/FruitsDataset/._.DS_Store  
   creating: FruitsDataset/watermelons/
   creating: FruitsDataset/apples/
  inflating: FruitsDataset/bananas/49.jpg  
  inflating: FruitsDataset/bananas/8.jpg  
  inflating: FruitsDataset/bananas/9.jpg  
  inflating: FruitsDataset/bananas/14.jpg  
  inflating: FruitsDataset/bananas/28.jpg  
  inflating: FruitsDataset/bananas/29.jpg  
  inflating: FruitsDataset/bananas/15.jpg  
  inflating: FruitsDataset/bananas/17.jpg  
  inflating: FruitsDataset/bananas/16.jpg  
  inflating: FruitsDataset/bananas/12.jpg  
  inflating: FruitsDataset/bananas/13.jpg  
  inflating: FruitsDataset/bananas/39.jpg  
  inflating: FruitsDataset/bananas/11.jpg  
  inflating: FruitsDataset/bananas/10.jpg  
  inflating: FruitsDataset/bananas/38.jpg  
  inflatin

In [ ]:
image_path = '/content/dataset'

In [ ]:
print(image_path)
labels = []
for i in os.listdir(image_path):
  if os.path.isdir(os.path.join(image_path, i)):
    labels.append(i)
print(labels)

/content/dataset
['onion', 'watermelon', 'banana', 'lemon', 'potato', 'apple']


In [ ]:
NUM_EXAMPLES = 5

for label in labels:
  label_dir = os.path.join(image_path, label)
  example_filenames = os.listdir(label_dir)[:NUM_EXAMPLES]
  fig, axs = plt.subplots(1, NUM_EXAMPLES, figsize=(10,2))
  for i in range(NUM_EXAMPLES):
    axs[i].imshow(plt.imread(os.path.join(label_dir, example_filenames[i])))
    axs[i].get_xaxis().set_visible(False)
    axs[i].get_yaxis().set_visible(False)
  fig.suptitle(f'Showing {NUM_EXAMPLES} examples for {label}')

plt.show()

In [ ]:
data = image_classifier.Dataset.from_folder(image_path)
train_data, remaining_data = data.split(0.9)
test_data, validation_data = remaining_data.split(0.5)

In [ ]:
spec = image_classifier.SupportedModels.EFFICIENTNET_LITE2
hparams = image_classifier.HParams(export_dir="exported_model",epochs=50)
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)

In [ ]:
model = image_classifier.ImageClassifier.create(
    train_data = train_data,
    validation_data = validation_data,
    options=options,
)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_3 (KerasLayer)  (None, 1280)              4869168   
                                                                 
 dropout_3 (Dropout)         (None, 1280)              0         
                                                                 
 dense_3 (Dense)             (None, 6)                 7686      
                                                                 
Total params: 4876854 (18.60 MB)
Trainable params: 7686 (30.02 KB)
Non-trainable params: 4869168 (18.57 MB)
_________________________________________________________________
None
Resuming from exported_model/checkpoint/model-0008
Epoch 1/50
216/216 [==============================] - 68s 299ms/step - loss: 0.7495 - accuracy: 0.9167 - val_loss: 0.6895 - val_accuracy: 1.0000
Epoch 2/50
216/216 [==============================] - 62s 288ms/step - loss

##Model Evaluation

In [ ]:
loss, acc = model.evaluate(test_data)
print(f'Test loss:{loss}, Test accuracy:{acc}')

1/1 [==============================] - 15s 15s/step - loss: 0.5965 - accuracy: 0.9167
Test loss:0.5965425372123718, Test accuracy:0.9166666865348816


##Model Conversion

In [ ]:
model.export_model()

In [69]:
!ls exported_model
files.download('exported_model/model.tflite')

checkpoint  metadata.json  model.tflite  summaries


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Retraining parameters

> Indented block



You can further customize how the retraining process runs to adjust training time and potentially increase the retrained model's performance. *These parameters are optional*. Use the `ImageClassifierModelOptions` class and the `HParams` class to set these additional options.

Use the `ImageClassifierModelOptions` class parameters to customize the existing model. It has the following customizable parameter that affects model accuracy:
* `dropout_rate`: The fraction of the input units to drop. Used in dropout layer. Defaults to 0.05.

Use the `HParams` class to customize other parameters related to training and saving the model:

* `learning_rate`: The learning rate to use for gradient descent training. Defaults to 0.001.
* `batch_size`: Batch size for training. Defaults to 2.
* `epochs`: Number of training iterations over the dataset. Defaults to 10.
* `steps_per_epoch`: An optional integer that indicates the number of training steps per epoch. If not set, the training pipeline calculates the default steps per epoch as the training dataset size divided by batch size.
* `shuffle`: True if the dataset is shuffled before training. Defaults to False.
* `do_fine_tuning`: If true, the base module is trained together with the classification layer on top. This defaults to False, which means only the classification layer is trained and pre-trained weights for the base module are frozen.
* `l1_regularizer`: A regularizer that applies a L1 regularization penalty. Defaults to 0.0.
* `l2_regularizer`: A regularizer that applies a L2 regularization penalty. Defaults to 0.0001.
* `label_smoothing`: Amount of label smoothing to apply. See [`tf.keras.losses`](https://www.tensorflow.org/api_docs/python/tf/keras/losses) for more details. Defaults to 0.1.
* `do_data_augmentation`: Whether or not the training dataset is augmented by applying random transformations such as cropping, flipping, etc. See [utils.image_preprocessing](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image) for details. Defaults to True.
* `decay_samples`: Number of training samples used to calculate the decay steps and create the training optimizer. Defaults to 2,560,000.
* `warmup_epochs`: Number of warmup steps for a linear increasing warmup schedule on the learning rate. Used to set up warmup schedule by `model_util.WarmUp`. Defaults to 2.



In [ ]:
hparams=image_classifier.HParams(epochs=15, export_dir="exported_model_2")
options = image_classifier.ImageClassifierOptions(supported_model=spec, hparams=hparams)
model_2 = image_classifier.ImageClassifier.create(
    train_data = train_data,
    validation_data = validation_data,
    options=options,
)

In [ ]:
loss, accuracy = model_2.evaluate(test_data)

In [ ]:
model.export_model()